# In-Context Learning using OpenAI API

In-context learning, also known as few-shot learning, is a powerful technique in natural language processing that leverages pretrained foundational models to perform complex tasks without the need for explicit retraining or parameter updates. This method enables models to quickly adapt to new tasks by using a limited number of input-output examples provided directly in the model's prompt. This approach helps guide the model in understanding how to process and respond to new types of inputs effectively.


Let’s have a look at the Banking-77 Dataset which contains online banking queries annotated with their corresponding intents

In [3]:
import pandas as pd

test = pd.read_csv('https://s.cleanlab.ai/banking-intent-50/test.csv')
examples_pool = pd.read_csv('https://s.cleanlab.ai/banking-intent-50/examples-pool.csv')

examples_pool[['text', 'label']].head()



,text,label
0,i moved to a new city and need to change my ad...,edit_personal_details
1,"on my transfer there was a ""decline"" message",declined_transfer
2,help! my wallet was stolen and someone is tak...,card_payment_fee_charged
3,"while abroad i got cash, and a wrong exchange ...",wrong_exchange_rate_for_cash_withdrawal
4,why can't i get cash?,getting_spare_card


### Building the Few-Shot Prompt


Prefix - List of Classes for Valid Completions
This text will go at the beginning of the prompt and will tell the LLM what the valid classes are so that it can consistently output a class. Without this, the LLM will not choose a valid class and output something not parsable.


Here we can also add an optional prefix that we will use later.


In [4]:
# Helper to get prefix for prompt. This gives the LLM all of the labels so that it chooses more accurately.
def get_prefix(examples_pool, prefix=""):
    s = ""
    if len(prefix) != 0:
        s += prefix
        s += '\n'
    s += "You can choose the label from: "
    classes = list(examples_pool.label.unique())
    s += ",".join(classes)
    return s

print(get_prefix(examples_pool, "Beware some labels in the examples may be noisy."))


Beware some labels in the examples may be noisy.
You can choose the label from: edit_personal_details,declined_transfer,card_payment_fee_charged,wrong_exchange_rate_for_cash_withdrawal,getting_spare_card,cash_withdrawal_charge,verify_top_up,transfer_timing,apple_pay_or_google_pay,card_payment_not_recognised,visa_or_mastercard,reverted_card_payment,transfer_not_received_by_recipient,country_support,wrong_amount_of_cash_received,refund_not_showing_up,card_linking,failed_transfer,exchange_via_app,fiat_currency_support,activate_my_card,direct_debit_payment_not_recognised,balance_not_updated_after_cheque_or_cash_deposit,cash_withdrawal_not_recognised,transfer_fee_charged,card_arrival,pending_top_up,extra_charge_on_statement,supported_cards_and_currencies,declined_card_payment,top_up_failed,automatic_top_up,transaction_charged_twice,disposable_card_limits,card_payment_wrong_exchange_rate,pending_transfer,declined_cash_withdrawal,balance_not_updated_after_bank_transfer,beneficiary_not_allowed

### Providing K-Shot Examples


Here we randomly choose 50 examples, 1 from each class to build a 50-shot prompt for the LLM.


In [5]:
# Helper method to get one example from each class for k-shot prompt.
import random
def get_examples(examples_pool):
    out = []
    unique_classes = examples_pool.label.unique()
    for i, cls in enumerate(unique_classes):
        temp = examples_pool[examples_pool.label==cls]
        random.seed(0+i)
        idx = random.choice(list(range(len(temp))))
        text = temp.iloc[idx].text
        label = temp.iloc[idx].label
        d = {'text':text, 'label':label}
        out.append(d)
    return out

examples = get_examples(examples_pool)
examples[1:5]

[{'text': 'it declined my transfer.', 'label': 'declined_transfer'},
 {'text': "why am i being charged for atm cash withdrawals? the only reason i use it is because it's been free! now you expect me to pay for them, and how much is that going to cost me?",
  'label': 'card_payment_fee_charged'},
 {'text': 'i attempted to get money using a foreign currency at an atm but the rate was highly inaccurate!',
  'label': 'wrong_exchange_rate_for_cash_withdrawal'},
 {'text': 'tell me where i can find the auto top up feature and a little bit about it please.',
  'label': 'getting_spare_card'}]

### Generating Entire Prompt


In [6]:
# ! pip install langchain-core
# ! pip install openai==0.28

In [7]:
# Helper to format the k-shot prompt with:
# - prefix
# - 1 example from each class
# - target text for classification
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts.few_shot import FewShotPromptTemplate
def get_prompt(examples_pool, text, examples, prefix=""):
    prompt_template = PromptTemplate(
        input_variables=["text", "label"],
        template="Text: {text}\nLabel: {label}",
    )

    p = FewShotPromptTemplate(
        example_prompt = prompt_template,
        examples = examples,
        prefix = get_prefix(examples_pool, prefix),
        suffix = "Text: {text}\nLabel:",
        input_variables = ['text'],
        )
    return p.format(text=text).strip()

print(get_prompt(examples_pool, "Classify this text!", examples, "Beware some labels in the examples may be noisy."))

Beware some labels in the examples may be noisy.
You can choose the label from: edit_personal_details,declined_transfer,card_payment_fee_charged,wrong_exchange_rate_for_cash_withdrawal,getting_spare_card,cash_withdrawal_charge,verify_top_up,transfer_timing,apple_pay_or_google_pay,card_payment_not_recognised,visa_or_mastercard,reverted_card_payment,transfer_not_received_by_recipient,country_support,wrong_amount_of_cash_received,refund_not_showing_up,card_linking,failed_transfer,exchange_via_app,fiat_currency_support,activate_my_card,direct_debit_payment_not_recognised,balance_not_updated_after_cheque_or_cash_deposit,cash_withdrawal_not_recognised,transfer_fee_charged,card_arrival,pending_top_up,extra_charge_on_statement,supported_cards_and_currencies,declined_card_payment,top_up_failed,automatic_top_up,transaction_charged_twice,disposable_card_limits,card_payment_wrong_exchange_rate,pending_transfer,declined_cash_withdrawal,balance_not_updated_after_bank_transfer,beneficiary_not_allowed

### Query OpenAI API


In [8]:
import openai, os
import string
# Helper method to prompt OpenAI LLM and get responses.
def get_response(prompt):
    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0,
      max_tokens=50,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )

    # Parse output to get just the label.
    resp = response['choices'][0]['text'].split('\n')[0].split(',')[0].strip().lower().rstrip(string.punctuation)

    # Just in case a respone is not a perfect match, we know.
    if resp not in examples_pool.label.unique():
        print(resp)
    return resp

text = "\'How can I change my pin?\'"
examples = get_examples(examples_pool)
prompt = get_prompt(examples_pool, text, examples)
response = get_response(prompt)
print("Model classified ", text, " as ", response)

AuthenticationError: No API key provided. You can set your API key in code using 'openai.api_key = <API-KEY>', or you can set the environment variable OPENAI_API_KEY=<API-KEY>). If your API key is stored in a file, you can point the openai module at it with 'openai.api_key_path = <PATH>'. You can generate API keys in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.